###Beolvasás

In [ ]:
#!pip install gtfs_functions
#!pip install KeplerGl
#!pip install keplergl jupyter
#!jupyter nbextension install --py --symlink --sys-prefix keplergl
#!jupyter nbextension enable --py --sys-prefix keplergl
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension
#!pip install gtfs_functions matplotlib basemap
#!pip install geopy
#!pip install pyshp
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
import gtfs_functions as gtfs
from gtfs_functions import Feed
import networkx as nx
import geopandas as gpd
import folium
from shapely.geometry import Point, LineString, Polygon, shape
from folium.plugins import MarkerCluster
#import keplergl as kp
from google.colab import output
from IPython.display import display
import pandas as pd
from gtfs_functions.gtfs_plots import map_gdf
import plotly.graph_objs as go
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from geopy.distance import geodesic
from math import radians, sin, cos, sqrt, atan2
from itertools import combinations
import numpy as np
import shapefile as shp
import seaborn as sns
import os
import copy
import fiona
import csv
import math
import colorsys
from difflib import SequenceMatcher
from folium.plugins import HeatMap
import branca.colormap as cmb
import re
from collections import defaultdict

In [ ]:
feed = Feed("budapest_gtfs.zip", time_windows=[6, 7]) #itt változtathatunk az időintervallumon
#feed = Feed("budapest_gtfs.zip")
routes = feed.routes
trips = feed.trips
stops = feed.stops
stop_times = feed.stop_times
shapes = feed.shapes

In [ ]:
segments_gdf = feed.segments
segments_gdf.head(2)
#segments_gdf.info()

In [ ]:
segments_freq = feed.segments_freq
segments_freq.head(2)
#print(type(segments_freq))

In [ ]:
colnames=['route_name', 'capacity']
df = pd.read_csv('capacity.csv', names=colnames, header=None)
df.head(2)

In [ ]:
segments_freq = segments_freq.merge(df, on='route_name')
segments_freq.head(2)

In [ ]:
seq_copy = copy.deepcopy(segments_freq)
seq_copy['weight'] = seq_copy['ntrips'] * seq_copy['capacity']

# Step 2: Normalize the weights for visualization
seq_copy['normalized_weight'] = seq_copy['weight'] / seq_copy['weight'].max()

# Step 3: Create a folium map centered on the mean coordinates of the data
mean_x, mean_y = seq_copy.geometry.centroid.x.mean(), seq_copy.geometry.centroid.y.mean()
m = folium.Map(location=[mean_y, mean_x], zoom_start=12)

# Step 4: Add the lines to the map
colormap = cmb.LinearColormap(colors=['blue', 'green', 'yellow', 'red'], vmin=0, vmax=1)
for _, row in seq_copy.iterrows():
    weight = row['normalized_weight']
    # Extracting coordinates directly
    locations = [(coord[1], coord[0]) for coord in row['geometry'].coords]
    folium.PolyLine(
        locations=locations,
        weight=weight * 10 + 0.1,  # Adjust thickness
        color=colormap(weight),  # Color based on weight
        opacity=0.7
    ).add_to(m)

# Step 5: Add colormap to the map
colormap.caption = 'Heatmap Intensity'
m.add_child(colormap)
m

###Multigráf

In [ ]:
def add_edges(G, gdf):
    for idx, row in gdf.iterrows():
        # Extract start and end coordinates
        start_coords = (row['geometry'].coords[0][1], row['geometry'].coords[0][0])
        end_coords = (row['geometry'].coords[-1][1], row['geometry'].coords[-1][0])

        # Calculate the product of ntrips and capacity
        ntrips_capacity = row['ntrips'] * row['capacity']

        # Create a unique key for the edge
        key = f"{row['route_name']}_{ntrips_capacity}"

        # Add edge to the graph with specified keys and additional attributes
        if row['direction_id'] == 0:
          G.add_edge(start_coords, end_coords, key=key, route_name=row['route_name'], ntrips_capacity=ntrips_capacity)
        elif row['direction_id'] == 1:
          G.add_edge(end_coords, start_coords, key=key, route_name=row['route_name'], ntrips_capacity=ntrips_capacity)

In [ ]:
G = nx.MultiDiGraph()
# Add edges from segments_freq to the graph
add_edges(G, segments_freq)

# Display basic information about the graph
#print(info(G))

In [ ]:
def create_folium_map(G):
    # Create a folium map centered around Budapest
    m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)

    # Add edges to the folium map with tooltips
    for u, v, data in G.edges(data=True):
        # Create the coordinates list for the PolyLine
        coords = [u, v]

        # Create the tooltip text
        tooltip_text = f"Route: {data['route_name']}<br>Capacity: {data['ntrips_capacity']}"

        # Add the PolyLine to the map with the tooltip
        folium.PolyLine(coords, color='blue', weight=2.5, opacity=1, tooltip=tooltip_text).add_to(m)

    # Add nodes to the folium map
    for node in G.nodes():
        folium.CircleMarker(location=node, radius=2, color='red').add_to(m)

    return m

In [ ]:
def create_folium_map_multicolor(G):
    # Create a folium map centered around Budapest
    m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)

    # Generate a list of distinct colors
    colors = list(mcolors.CSS4_COLORS.values())
    max_colors = min(len(colors), 50)  # Limit to 30 distinct colors if there are fewer colors available
    colors = colors[:max_colors]

    # Create a dictionary to track the number of parallel edges
    edge_count = {}

    # Add edges to the folium map with tooltips
    for u, v, key, data in G.edges(keys=True, data=True):
        # Create the coordinates list for the PolyLine
        coords = [u, v]

        # Create a unique identifier for the edge pair (u, v)
        edge_id = (u, v)
        if edge_id not in edge_count:
            edge_count[edge_id] = 0
        edge_index = edge_count[edge_id]
        edge_count[edge_id] += 1

        # Assign a color based on the edge index
        color = colors[edge_index % len(colors)]

        # Create the tooltip text
        tooltip_text = f"Route: {data['route_name']}<br>Capacity: {data['ntrips_capacity']}"

        # Add the PolyLine to the map with the tooltip
        folium.PolyLine(coords, color=color, weight=2.5, opacity=1, tooltip=tooltip_text).add_to(m)

    # Add nodes to the folium map
    for node in G.nodes():
        folium.CircleMarker(location=node, radius=2, color='red').add_to(m)

    return m

In [ ]:
s = create_folium_map(G)#_multicolor(G)

# Display the map
#m.save("multidigraph_map.html")
s

In [ ]:
edges_gdf = gpd.GeoDataFrame(columns=['geometry', 'route_name', 'ntrips'])
for u, v, data in G.edges(data=True):
    data['geometry'] = LineString([u, v])
    pd.concat([edges_gdf, gpd.GeoDataFrame([data])])

# Initialize KeplerGl map
m = kp.KeplerGl(height=400)

# Add data to KeplerGl map
m.add_data(data=edges_gdf, name='Segments')

# Configure map layers and tooltips
m.config = {
#config = {
    "version": "v1",
    "config": {
        "mapState": {
            "latitude": 47.4979,
            "longitude": 19.0402,
            "zoom": 12
        },
        "mapStyle": {
            "styleType": "dark"
        },
        "visState": {
            "layers": [
                {
                    "id": "Segments",
                    "type": "line",
                    "config": {
                        "dataId": "Segments",
                        "label": "Segments",
                        "columns": {
                            "geojson": "geometry"
                        },
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.8,
                            "thickness": 1,
                            "colorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300"
                                ]
                            },
                            "strokeColor": [170, 255, 0],
                            "strokeWidth": 0.5
                        },
                        "textLabel": [
                            {
                                "field": {
                                    "name": "route_name"
                                },
                                "color": [255, 255, 255],
                                "size": 18,
                                "offset": [0, 0],
                                "anchor": "start",
                                "alignment": "center"
                            }
                        ]
                    }
                }
            ]
        }
    }
}

# Display the map (this would work in a Jupyter notebook or similar environment)
m
output.enable_custom_widget_manager()

###Sima gráf

In [ ]:
def add_edges_with_weights(G, gdf):
    for idx, row in gdf.iterrows():
        # Extract start and end coordinates
        start_coords = (row['geometry'].coords[0][1], row['geometry'].coords[0][0])
        end_coords = (row['geometry'].coords[-1][1], row['geometry'].coords[-1][0])

        ntrips_capacity = row['ntrips'] * row['capacity']

        # Add edge to the graph with ntrips as weight
        if row['direction_id'] == 0:
          G.add_edge(start_coords, end_coords, weight=ntrips_capacity, attr_dict=row.to_dict())
        elif row['direction_id'] == 1:
          G.add_edge(end_coords, start_coords, weight=ntrips_capacity, attr_dict=row.to_dict())

In [ ]:
def add_edges_with_weights2(G, gdf):
    for idx, row in gdf.iterrows():
        # Extract start and end coordinates
        start_coords = (row['geometry'].coords[0][1], row['geometry'].coords[0][0])
        end_coords = (row['geometry'].coords[-1][1], row['geometry'].coords[-1][0])

        ntrips_capacity = row['ntrips'] * row['capacity']

        if row['direction_id'] == 0:
            if G.has_edge(start_coords, end_coords):
                # If the edge exists, update the weight
                G[start_coords][end_coords]['weight'] += ntrips_capacity
                G[start_coords][end_coords]['ntrips_capacity'] += ntrips_capacity
            else:
                # If the edge does not exist, add it with the new weight and attributes
                G.add_edge(start_coords, end_coords, weight=ntrips_capacity, route_name=row['route_name'], ntrips_capacity=ntrips_capacity)
        elif row['direction_id'] == 1:
            if G.has_edge(end_coords, start_coords):
                # If the edge exists, update the weight
                G[end_coords][start_coords]['weight'] += ntrips_capacity
                G[end_coords][start_coords]['ntrips_capacity'] += ntrips_capacity
            else:
                # If the edge does not exist, add it with the new weight and attributes
                G.add_edge(end_coords, start_coords, weight=ntrips_capacity, route_name=row['route_name'], ntrips_capacity=ntrips_capacity)

In [ ]:
G2 = nx.DiGraph()
add_edges_with_weights2(G2, segments_freq)

In [ ]:
def create_folium_map2(G):
    # Create a folium map centered around Budapest
    m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)

    # Add edges to the folium map with tooltips
    for u, v, data in G.edges(data=True):
        # Create the coordinates list for the PolyLine
        coords = [u, v]

        # Create the tooltip text
        capacity = data.get('ntrips_capacity', 'N/A')
        tooltip_text = f"Capacity: {capacity}"

        # Add the PolyLine to the map with the tooltip
        folium.PolyLine(coords, color='blue', weight=2.5, opacity=1, tooltip=tooltip_text).add_to(m)

    # Add nodes to the folium map
    for node in G.nodes():
        folium.CircleMarker(location=node, radius=2, color='red').add_to(m)

    return m

In [ ]:
def create_folium_heatmap(G):
    # Create a folium map centered around Budapest
    m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)

    # Normalize the edge weights for color scaling
    edge_weights = [data.get('ntrips_capacity', 1) for u, v, data in G.edges(data=True)]
    min_weight = min(edge_weights)
    max_weight = max(edge_weights)

    def get_color(weight):
        # Normalize the weight to the range [0, 1]
        norm_weight = (weight - min_weight) / (max_weight - min_weight) if max_weight != min_weight else 0.5
        # Get a color from the 'rainbow' colormap
        color = cm.rainbow(norm_weight)
        # Convert color to hex
        return mcolors.to_hex(color)

    # Add edges to the folium map with tooltips
    for u, v, data in G.edges(data=True):
        # Create the coordinates list for the PolyLine
        coords = [u, v]

        # Get the weight and color for the edge
        capacity = data.get('ntrips_capacity', 1)
        color = get_color(capacity)

        # Create the tooltip text
        tooltip_text = f"Capacity: {capacity}"

        # Add the PolyLine to the map with the tooltip
        folium.PolyLine(coords, color=color, weight=2.5, opacity=1, tooltip=tooltip_text).add_to(m)

    # Add nodes to the folium map
    for node in G.nodes():
        folium.CircleMarker(location=node, radius=2, color='gray').add_to(m)

    return m

In [ ]:
r = create_folium_heatmap(G2)
r

In [ ]:
r = create_folium_map2(G2)
r

In [ ]:
edges_data = []
for u, v, data in G.edges(data=True):
    edges_data.append({
        'geometry': LineString([u, v]),
        'route_name': data['route_name'],
        'ntrips': data['ntrips']
    })

edges_gdf = gpd.GeoDataFrame(edges_data, columns=['geometry', 'route_name', 'ntrips'])

# Initialize KeplerGl map
m = kp.KeplerGl(height=600)

# Add data to KeplerGl map
m.add_data(data=edges_gdf, name='Segments', replace=True)

# Configure map layers and tooltips
m.config = {
    "version": "v1",
    "config": {
        "mapState": {
            "latitude": 47.4979,
            "longitude": 19.0402,
            "zoom": 12
        },
        "mapStyle": {
            "styleType": "dark"
        },
        "visState": {
            "layers": [
                {
                    "id": "Segments",
                    "type": "line",
                    "config": {
                        "dataId": "Segments",
                        "label": "Segments",
                        "columns": {
                            "geojson": "geometry"
                        },
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.8,
                            "thickness": 1,
                            "colorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300"
                                ]
                            },
                            "strokeColor": [170, 255, 0],
                            "strokeWidth": 0.5
                        },
                        "textLabel": [
                            {
                                "field": {
                                    "name": "route_name"
                                },
                                "color": [255, 255, 255],
                                "size": 18,
                                "offset": [0, 0],
                                "anchor": "start",
                                "alignment": "center"
                            }
                        ]
                    }
                }
            ]
        }
    }
}

# Display the map (this would work in a Jupyter notebook or similar environment)
m

In [ ]:
def add_edges2(G, gdf): #egyelőre nem kell
    for idx, row in gdf.iterrows():
        # Extract start and end coordinates
        start_coords = (row['geometry'].coords[0][1], row['geometry'].coords[0][0])
        end_coords = (row['geometry'].coords[-1][1], row['geometry'].coords[-1][0])

        # Create a unique key for the edge
        key = f"{row['route_name']}_{row['ntrips']}"

        # Add nodes with stop names
        G.add_node(start_coords, stop_name=row['start_stop_name'])
        G.add_node(end_coords, stop_name=row['end_stop_name'])

        # Add edge to the graph with specified keys
        G.add_edge(start_coords, end_coords, key=key, route_name=row['route_name'], ntrips=row['ntrips'])

In [ ]:
G3 = nx.MultiDiGraph()
add_edges2(G3, segments_freq)

In [ ]:
def add_edgesline(m, gdf):
    for idx, row in gdf.iterrows():
        if isinstance(row['geometry'], LineString):
            coords = [(pt[1], pt[0]) for pt in row['geometry'].coords]
            tooltip_text = f"Route: {row['route_name']}<br>Trips: {row['ntrips']}<br>From: {row['start_stop_name']}<br>To: {row['end_stop_name']}"
            folium.PolyLine(coords, color='blue', weight=2.5, opacity=0.8, tooltip=tooltip_text).add_to(m)

In [ ]:
m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)
add_edgesline(m, segments_freq)

# Save the map to an HTML file
#m.save('budapest_transit_map.html')

# Display the map
m

In [ ]:
lines = []
for idx, row in segments_freq.iterrows():
    if isinstance(row['geometry'], LineString):
        coords = list(row['geometry'].coords)
        lat, lon = zip(*coords)
        lines.append(go.Scattermapbox(
            lat=lat,
            lon=lon,
            mode='lines',
            line=dict(width=2, color='blue'),
            text=f"Route: {row['route_name']}<br>Trips: {row['ntrips']}<br>From: {row['start_stop_name']}<br>To: {row['end_stop_name']}",
            hoverinfo='text'
        ))

# Create a Plotly figure
fig = go.Figure()

fig.add_traces(lines)

fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_zoom=12,
    mapbox_center={"lat": 47.4979, "lon": 19.0402},
    height=600
)

fig.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
m = Basemap(projection='merc', llcrnrlat=47.0, urcrnrlat=48.0, llcrnrlon=18.5, urcrnrlon=19.5, resolution='i', ax=ax)
m.drawcoastlines()
m.drawcountries()
m.drawmapboundary(fill_color='aqua')
m.fillcontinents(color='lightgreen', lake_color='aqua')
m.drawparallels(range(47, 49), labels=[1,0,0,0])
m.drawmeridians(range(18, 20), labels=[0,0,0,1])

# Add segments to the map
for idx, row in segments_freq.iterrows():
    if isinstance(row['geometry'], LineString):
        coords = list(row['geometry'].coords)
        lons, lats = zip(*coords)
        x, y = m(lons, lats)
        m.plot(x, y, marker=None, color='b', linewidth=2, label=f"{row['route_name']} ({row['ntrips']} trips)")

# Add a legend
plt.legend(loc='lower left', fontsize='small')

plt.title('Budapest Public Transport Segments')
plt.show()

###Csúcsösszevonás 1

In [ ]:
# Assuming 'stops' is the GeoDataFrame and has columns 'stop_name', 'stop_id', 'geometry'
stops_list = []

# Step 1: Extract necessary data into a list of [stop_name, stop_id, coordinate]
for _, row in stops.iterrows():
    stop_name = row['stop_name']
    stop_id = row['stop_id']
    coordinate = (row['stop_lon'], row['stop_lat'])
    stops_list.append([stop_name, stop_id, coordinate])

In [ ]:
def remove_brackets_and_extras(stop_name):
    stop_name = re.sub(r'\(.*?\)', '', stop_name)  # Remove text in parentheses
    stop_name = re.sub(r'\[.*?\]', '', stop_name)  # Remove text in square brackets
    stop_name = re.sub(r'\bH\b', '', stop_name)    # Remove standalone 'H'
    stop_name = re.sub(r'\bM\b', '', stop_name)    # Remove standalone 'M'
    stop_name = stop_name.replace('+', '')         # Remove the '+' sign
    return stop_name.strip()

In [ ]:
# Clean stop names in the stops_list
for stop in stops_list:
    stop[0] = remove_brackets_and_extras(stop[0])

In [ ]:
# Step 3: Create a dictionary with stop_name as keys and lists of [stop_id, coordinate] as values
stops_dict = defaultdict(list)
for stop_name, stop_id, coord in stops_list:
    stops_dict[stop_name].append([stop_id, coord])

In [ ]:
def haversine(coord1, coord2):
    # Radius of the Earth in meters
    R = 6371000

    lat1, lon1 = coord1
    lat2, lon2 = coord2

    # Convert latitude and longitude from degrees to radians
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    # Haversine formula
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance in meters
    distance = R * c
    return distance

In [ ]:
# Step 4: Group by coordinates within each stop name
def group_coordinates_by_distance(entries, threshold=600):
    groups = []
    for stop_id, coord in entries:
        added = False
        for group in groups:
            if all(haversine(coord, existing_coord[1]) < threshold for existing_coord in group):
                group.append([stop_id, coord])
                added = True
                break
        if not added:
            groups.append([[stop_id, coord]])
    return groups

# Create a list to hold all groups of stop_ids and coordinates
grouped_by_stop = []

# Group coordinates for each stop name
for stop_name, entries in stops_dict.items():
    groups = group_coordinates_by_distance(entries)
    grouped_by_stop.extend(groups)

In [ ]:
# Step 5: Calculate average coordinates for each group and replace them in the grouped list
def calculate_average_coordinates(groups):
    for group in groups:
        if not group:
            continue
        avg_lat = sum(coord[1][1] for coord in group) / len(group)
        avg_lon = sum(coord[1][0] for coord in group) / len(group)
        average_coord = (avg_lon, avg_lat)
        for item in group:
            item[1] = average_coord

# Apply the average calculation to all groups
calculate_average_coordinates(grouped_by_stop)

In [ ]:
# The resulting grouped_by_stop list contains lists of [stop_id, average_coordinate]
#for group in grouped_by_stop:
    #print(group)


In [ ]:
"""with open('dict.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in filtered_stop_coords_dict.items():
       writer.writerow([key, value])"""

In [ ]:
# Create a dictionary to store stop_id as keys and average coordinates as values
average_coords_dict = {}

# Iterate through the grouped_by_stop list to fill the dictionary
for group in grouped_by_stop:
    for stop_id, avg_coord in group:
        average_coords_dict[stop_id] = avg_coord

In [ ]:
# Function to update the geometry of each row
def update_geometry(row):
    # Get the original LineString
    line = row['geometry']

    # Look up the average coordinates using the stop IDs
    start_coord = average_coords_dict.get(row['start_stop_id'])
    end_coord = average_coords_dict.get(row['end_stop_id'])

    # If coordinates are found, update the LineString
    if start_coord and end_coord:
        # Create a new LineString with updated start and end coordinates
        new_coords = [start_coord] + list(line.coords[1:-1]) + [end_coord]
        return LineString(new_coords)
    else:
        # If not found, return the original LineString
        return line

seq_copy = copy.deepcopy(segments_freq)
seq_copy['geometry'] = seq_copy.apply(update_geometry, axis=1)

In [ ]:
G4 = nx.DiGraph()
add_edges_with_weights2(G4, seq_copy)
#G4.nodes()
r = create_folium_map2(G4)
r

In [ ]:
def add_edges_with_weights3(G, gdf):
    for idx, row in gdf.iterrows():
        # Extract start and end coordinates
        start_coords = (row['start_new_coord'][0], row['start_new_coord'][1])
        end_coords = (row['end_new_coord'][0], row['end_new_coord'][1])
        #print(start_coords)
        #print(end_coords)
        ntrips_capacity = row['ntrips'] * row['capacity']

        if row['direction_id'] == 0:
            if G.has_edge(start_coords, end_coords):
                # If the edge exists, update the weight
                G[start_coords][end_coords]['weight'] += ntrips_capacity
                G[start_coords][end_coords]['ntrips_capacity'] += ntrips_capacity
            else:
                # If the edge does not exist, add it with the new weight and attributes
                G.add_edge(start_coords, end_coords, weight=ntrips_capacity, route_name=row['route_name'], ntrips_capacity=ntrips_capacity)
        elif row['direction_id'] == 1:
            if G.has_edge(end_coords, start_coords):
                # If the edge exists, update the weight
                G[end_coords][start_coords]['weight'] += ntrips_capacity
                G[end_coords][start_coords]['ntrips_capacity'] += ntrips_capacity
            else:
                # If the edge does not exist, add it with the new weight and attributes
                G.add_edge(end_coords, start_coords, weight=ntrips_capacity, route_name=row['route_name'], ntrips_capacity=ntrips_capacity)

In [ ]:
def create_folium_map3(G):
     # Create a folium map centered around Budapest
    m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)

    # Add edges to the folium map with tooltips
    for u, v, data in G.edges(data=True):
        # Create the coordinates list for the PolyLine
        coords = [u, v]

        # Create the tooltip text
        capacity = data.get('ntrips_capacity', 'N/A')
        tooltip_text = f"Capacity: {capacity}"

        # Add the PolyLine to the map with the tooltip
        folium.PolyLine(coords, color='blue', weight=2.5, opacity=1, tooltip=tooltip_text).add_to(m)

        # Debugging statement to verify coordinates being added to the map
        #print(f"Adding line from {u} to {v} with capacity: {capacity}")

    # Add nodes to the folium map
    for node in G.nodes():
        folium.CircleMarker(location=node, radius=2, color='red').add_to(m)
        # Debugging statement to verify nodes being added to the map
        #print(f"Adding node at location: {node}")

    return m

In [ ]:
G5 = nx.DiGraph()
add_edges_with_weights3(G5, seq_copy)
G5.nodes()

In [ ]:
r = create_folium_map3(G5)
r

###Zónák és OD mátrix

In [ ]:
sf = shp.Reader("zone_zone.shp")
#print(type(sf))

In [ ]:
def read_shapefile_df(sf):
    """
    Read a shapefile into a Pandas dataframe with a 'coords'
    column holding the geometry information. This uses the pyshp
    package
    """
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df

In [ ]:
def read_shapefile_gdf(sf):
    """
    Read a shapefile into a GeoDataFrame.
    """
    # Extract the fields and records
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()

    # Extract the shapes and convert them to shapely geometries
    geometries = []
    for shape in sf.shapes():
        if len(shape.points) == 1:  # Point
            geometries.append(Point(shape.points[0]))
        else:  # Polygon
            geometries.append(Polygon(shape.points))

    # Create a GeoDataFrame
    df = gpd.GeoDataFrame(columns=fields, data=records)
    df = df.assign(geometry=geometries)

    return df

In [ ]:
df = read_shapefile_gdf(sf)
df

In [ ]:
def plot_map(sf, df, x_lim=None, y_lim=None, figsize=(17, 11), filtered_ids=None):
    '''
    Plot map with lim coordinates and colored shapes based on 'NO' attribute.
    '''
    plt.figure(figsize=figsize)
    id = 0

    cmap = plt.get_cmap('viridis')
    norm = plt.Normalize(df['NO'].min(), df['NO'].max())

    for shape in sf.shapeRecords():
        if filtered_ids is not None and id not in filtered_ids:
            id += 1
            continue
        x = [i[0] for i in shape.shape.points[:]]
        y = [i[1] for i in shape.shape.points[:]]
        color = cmap(norm(df.loc[id, 'NO']))
        plt.fill(x, y, color=color, edgecolor='k')

        if x_lim is None and y_lim is None:
            x0 = np.mean(x)
            y0 = np.mean(y)
            plt.text(x0, y0, id, fontsize=10)
        id += 1

    if x_lim is not None and y_lim is not None:
        plt.xlim(x_lim)
        plt.ylim(y_lim)

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    plt.colorbar(sm, orientation='vertical', label='NO Attribute')

    plt.show()


In [ ]:
# Filter dataframe based on 'NO' attribute
filtered_df = df[(df['NO'] >= 1000) & (df['NO'] <= 23999)]
filtered_ids = filtered_df.index.tolist()

print(filtered_df.shape)  # Print the shape of the filtered dataframe

# Plot the filtered map with colors
plot_map(sf, filtered_df, filtered_ids=filtered_ids)

In [ ]:
filtered_df
print(type(filtered_df))

In [ ]:
geom = copy.deepcopy(segments_freq)
geom.head(2)

In [ ]:
if geom.crs is None:
    # Replace 'EPSG:4326' with the correct CRS if known
    geom.set_crs(epsg=4326, inplace=True)

# Define the path to the shapefile
shapefile_path = 'zone_zone.SHP'

# Check if the shapefile exists
if not os.path.exists(shapefile_path):
    raise FileNotFoundError(f"Shapefile not found: {shapefile_path}")

# Load the shapefile using Fiona
with fiona.open(shapefile_path) as shapefile:
    # Ensure shapefile has a CRS
    if shapefile.crs is None:
        # Replace 'EPSG:4326' with the correct CRS if known
        shapefile_crs = 'EPSG:4326'
    else:
        shapefile_crs = shapefile.crs

    # Convert the shapefile features to GeoDataFrame
    shapefile_gdf = gpd.GeoDataFrame.from_features(shapefile, crs=shapefile_crs)

    # Filter the shapefile GeoDataFrame based on the 'NO' attribute
    shapefile_filtered = shapefile_gdf[(shapefile_gdf['NO'] >= 1000) & (shapefile_gdf['NO'] <= 23999)]

    # Ensure CRS match
    if shapefile_filtered.crs != geom.crs:
        shapefile_filtered = shapefile_filtered.to_crs(geom.crs)

    # Initialize the new columns with None values
    geom['first_coord_shape_index'] = None
    geom['second_coord_shape_index'] = None

    # Convert filtered shapes to Shapely geometries
    filtered_shapes = [(shape(geom_shape), shape_id) for geom_shape, shape_id in zip(shapefile_filtered.geometry, shapefile_filtered.index)]

    # Iterate through the rows of the GeoDataFrame
    for idx, row in geom.iterrows():
        # Extract the coordinates from the geometry column
        coords = list(row.geometry.coords)
        first_coord = Point(coords[0])
        second_coord = Point(coords[-1])
        """if idx <= 10:
          print(coords)
          print(first_coord)
          print(second_coord)"""

        # Check which shape the first coordinate falls within
        first_match = next((shape_id for geom_shape, shape_id in filtered_shapes if geom_shape.contains(first_coord)), None)
        geom.at[idx, 'first_coord_shape_index'] = first_match

        # Check which shape the second coordinate falls within
        second_match = next((shape_id for geom_shape, shape_id in filtered_shapes if geom_shape.contains(second_coord)), None)
        geom.at[idx, 'second_coord_shape_index'] = second_match

In [ ]:
geom.head(2)

In [ ]:
geom.drop(columns=['route_id', 'direction_id', 'segment_name', 'window', 'min_per_trip', 'start_stop_id', 'end_stop_id', 'segment_id', 'geometry']).to_csv('geom.csv')
from google.colab import files
files.download('geom.csv')

In [ ]:
# Initialize the matrix
matrix_size = len(shapefile_filtered)
matrix_cap = np.zeros((matrix_size, matrix_size))

# Populate the matrix based on geom rows
for idx, row in geom.iterrows():
    first_index = row['first_coord_shape_index']
    second_index = row['second_coord_shape_index']
    if first_index is not None and second_index is not None and first_index <= 921 and second_index <= 921:
      matrix_cap[first_index, second_index] += row['ntrips'] * row['capacity']
matrix_cap

In [ ]:
matrix_trip = np.zeros((matrix_size, matrix_size))
for idx, row in geom.iterrows():
    first_index = row['first_coord_shape_index']
    second_index = row['second_coord_shape_index']
    if first_index is not None and second_index is not None and first_index <= 921 and second_index <= 921:
      matrix_trip[first_index, second_index] += row['ntrips']
matrix_trip